In [1]:
!pip install langchain-community==0.0.11 pypdf==3.17.4 langchain==0.1.0 python-dotenv==1.0.0 langchain-openai==0.0.2.post1 faiss-cpu==1.7.4 tiktoken==0.5.2 langchainhub==0.1.14 pip sentence-transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
inference_server_url = "http://hf-tgi.llm-hosting.svc.cluster.local:3000/"
index_name = "docs"
schema_name = "redis_schema.yaml"

In [5]:
from langchain import hub
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.redis import Redis
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

def main():

  load_dotenv()
  loader = PyPDFDirectoryLoader('pdf/')  
  documents = loader.load()

  text_splitter = CharacterTextSplitter(
        chunk_size=1000, chunk_overlap=50, separator="\n")
  docs = text_splitter.split_documents(documents)
      
  retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

  llm = Ollama(
    base_url="http://ollama.redis.svc.cluster.local:11434",
    model="mistral",
    callbacks=[StreamingStdOutCallbackHandler()])  

  combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
  combine_docs_chain.invoke({"context": docs, "input": "What is OpenShift AI ?"})

if __name__ == "__main__":
  main()

 OpenShift AI is a Red Hat product that provides a platform for deploying, managing, and scaling machine learning (ML) and deep learning (DL) workloads using containers in the Red Hat OpenShift Container Platform. It allows data scientists and ML engineers to build, train, and deploy ML models in a collaborative, scalable, and secure environment, while also integrating with other tools and services in the OpenShift ecosystem for data processing, model serving, and more.